In [0]:
#las clases de fechas se utilizan principalmente para dar formato a la fecha de ejecucion y para los logs
from datetime import date, datetime
#las librerias de pySpark se utilizan en el proceso de lectura de silver y guardado de tablas delta
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [0]:
#se crea un widget para escribir la fecha de ejecucion ondemand y recivir el parametro para su ejecucion programada
dbutils.widgets.text(name='fec_ejecucion', defaultValue='', label='fecha de ejecucion')
txt_fec_ejecucion = dbutils.widgets.get(name='fec_ejecucion')

In [0]:
#se formatea la fecha de ejecucion obtenida y se separa para generar los logs correspondientes
today = datetime.strptime(txt_fec_ejecucion, "%Y-%m-%d")
year = str(today.year)
month = str(today.month).zfill(2)
day = str(today.day).zfill(2)

In [0]:
#variables principalmente para la lectura de tabla silver y carga de gold ademas de algunas declaraciones para logs
stage_id = "002"
zone = "GOLD"
tbl = "FT_ADMISION"
gold_tbl_name = "{}".format(tbl)
gold_routhe = "dbfs:/mnt/lakehouse-gold/ORACLE/"
log_routhe = "dbfs:/mnt/lakehouse-gold/log/"+gold_tbl_name
schema = "gold."
start_process = datetime.now()
start_process_time = start_process.strftime("%Y-%m-%d %H:%M:%S")

In [0]:
#generacion script sql para lectura de capa silver y generacion de tabla gold
#el COD_CURRICULUM 280004 antes se homologaba al codigo 2801, ahora se homologa al 9999 porque ahora existe el codigo homologado para un postgrado, ademas confirmamos que deben cruzar las id_carrera no el homologado pues esta homologacion es solo para reporte
#estamos a espera de confirmacion si el cruce entre las primeras 2 tablas se mantiene como left join o se cambia a inner join(ficha_alumno -> alumno)

query_q = f"""
select distinct
f.COD_FICHA_ALUMNO AS ID_FICHA_ALUMNO,
coalesce(cole.ID_COLEGIO,-1) AS ID_COLEGIO,
coalesce(a.COD_PERS,-1) AS ID_PERSONA,
coalesce(car.COD_CARRERA,-1) AS ID_CARRERA,
coalesce(cu.COD_CURRICULUM,-1) AS ID_CURRICULUM,
coalesce(car.COD_CAMPUS,-1) AS ID_CAMPUS,
coalesce(unac.COD_FACULTAD,-1) AS ID_FACULTAD,
coalesce(a.COD_NOM_VIA_ADMISION,-1) AS ID_VIA_ADMISION,
CAST(coalesce(det_via.COD_PARAM,-1) AS INT) AS ID_DETALLE_VIA_ADMISION,
Case
    when car.COD_CARRERA = 301 then 201
    when car.COD_CARRERA = 501 then 500
    when car.COD_CARRERA = 600 then 610
    when car.COD_CARRERA = 620 and f.ANIO_FICHA < 2018 then 610
    when car.COD_CARRERA = 1110 then 1100
    when car.COD_CARRERA = 1001 then 800
    when car.COD_CARRERA in(1390,1391) then 1300
    when car.COD_CARRERA in(3800,3803) then 3830
    when car.COD_CARRERA = 7000 then 7002
    when car.COD_CARRERA = 7001 then 7003
    when car.COD_CARRERA in(9101,9104,9201) then 9204
    when a.COD_CURRICULUM = 280004 then 9999
    when car.COD_CARRERA = 6410 then 6417
    when car.COD_CARRERA = 6413 then 6418
    when car.COD_CARRERA = 9 then 2076 else coalesce(car.COD_CARRERA,-1)
end ID_CARRERA_HOMOLOGA,
hc.CLAVE_NACIONAL AS ID_CARRERA_DEMRE,
f.ANIO_EGRESO_COLEGIO,
f.ANIO_FICHA,
f.COD_ALUMNO,
f.COD_NIVEL_EDUCACIONAL_MADRE,
f.COD_NIVEL_EDUCACIONAL_PADRE,
f.COD_PREFERENCIA_POSTULACION,
case
    when f.COD_PREFERENCIA_POSTULACION = 1 then "Primera Preferencia"
    when f.COD_PREFERENCIA_POSTULACION = 2 then "Segunda Preferencia"
    when f.COD_PREFERENCIA_POSTULACION >= 3 then "Tercera (o más) Preferencia"
    else "Sin Información"
end as DESC_PREFERENCIA_POSTULACION,
f.COD_CON_QUIEN_VIVE,
f.COD_PERIODO_FICHA AS COD_SEMESTRE_ADMISION,
f.PROM_PAA AS PROM_LENGUAJE_MATEMATICAS,
f.PRUEBA_PAA_HISTORIA,
f.PRUEBA_PAA_MATEM,
f.PRUEBA_PAA_VERBAL,
f.PRUEBA_PCE_BIOLOGIA,
f.PRUEBA_PCE_CSS,
f.PRUEBA_PCE_FISICA,
f.PRUEBA_PCE_MATEM,
f.PRUEBA_PCE_QUIM,
f.PRUEBA_PAES_CIENC,
f.PRUEBA_PAES_CLEC,
f.PRUEBA_PAES_HISCS,
f.PRUEBA_PAES_MATM1,
f.PRUEBA_PAES_MATM2,
f.PRUEBA_PSU_LENG_COMUNIC,
f.PRUEBA_PSU_MATEM,
f.PRUEBA_PSU_HIST_CSS,
f.PRUEBA_PSU_CIENCIA,
f.PRUEBA_ESPECIAL,
f.PTJE_NOTA_ENSEN_MEDIA,
f.PTJE_POND_SELEC,
f.PTJE_RANKING,
f.PUESTO,
coalesce(a.NRO_ALUMNO,"-1") AS NUM_ALUMNO,
CASE  WHEN f.ANIO_EGRESO_COLEGIO <= 1900 or f.ANIO_EGRESO_COLEGIO is null then "Sin Información"
      WHEN f.ANIO_FICHA-f.ANIO_EGRESO_COLEGIO = 1 then "De la Promoción"
      ELSE "Rezagados" 
End AS DESC_ANIO_EGRESO_ENSENANZA_MEDIA,
CASE
    WHEN via_ad.NOM_PARAM = 'ORDINARIA PAA' THEN 'Admisión Centralizada'
    WHEN det_via.NOM_PARAM IN ('BECA EXCELENCIA ACADEMICA', 'TALENTOS E INCLUSION', 'INCLUSIÓN VILLARRICA', 'PACE', 'EQUIDAD DGÉNERO', 'INTERCULTURALIDAD', 'PERSONAS CON DISCAPACIDAD') THEN 'Admisión Equidad'
    WHEN via_ad.NOM_PARAM = 'COMPLEMENTARIA' THEN 'Admisión Especial o Complementaria'
    ELSE 'Otras Vías Admisión Especial'
END AS DESC_VIA_ADMISION_EQUIDAD,
CASE
    WHEN via_ad.NOM_PARAM = 'ORDINARIA PAA' THEN 'Admisión Centralizada'
    WHEN det_via.NOM_PARAM = 'BECA EXCELENCIA ACADEMICA' THEN 'BEA'
    WHEN det_via.NOM_PARAM IN ('TALENTOS E INCLUSION', 'INCLUSIÓN VILLARRICA') THEN 'Talento e Inclusión'
    WHEN det_via.NOM_PARAM = 'PACE' THEN 'PACE'
    WHEN det_via.NOM_PARAM = 'EQUIDAD DE GÉNERO' AND car.COD_UNIDAD_ACADEM = 38 THEN 'Equidad de Género Teología'
    WHEN det_via.NOM_PARAM = 'EQUIDAD DE GÉNERO' AND car.COD_UNIDAD_ACADEM != 38 THEN 'Cupo Mujeres en Ciencia'
    WHEN det_via.NOM_PARAM = 'INTERCULTURALIDAD' THEN 'Intercultural'
    WHEN det_via.NOM_PARAM = 'PERSONAS CON DISCAPACIDAD' THEN 'Necesidades Especiales'
    WHEN via_ad.NOM_PARAM = 'COMPLEMENTARIA' THEN 'Admisión Especial o Complementaria'
    ELSE 'Otras Vías Admisión Especial'
END AS DESC_DETALLE_VIA_ADMISION,
Case 
    WHEN deped.NOM_PARAM = "" or deped.NOM_PARAM ="NO TIENE" or  deped.NOM_PARAM is null then "Sin Información"
    WHEN deped.NOM_PARAM in("CORPORACIÓN DE ADMINISTRACIÓN DELEGADA","CORPORACIÓN MUNICIPAL","MUNICIPAL","SERVICIO LOCAL DE EDUCACIÓN (SLE)","SUBVENCIONADO GRATUITO") then "Público (Mun + SLE)"
    WHEN deped.NOM_PARAM in("FINANCIAMIENTO COMPARTIDO","PARTICULAR SUBVENCIONADO") then "Particular Subvencionado"
    WHEN deped.NOM_PARAM in("PAGADO","PARTICULAR NO SUBVENCIONADO") then "Particular Pagado"
    ELSE "Sin Información" 
END AS DESC_DEPENDENCIA_EDUCACIONAL,
Case 
    WHEN reg.DESC_REGION_ABREV = "" or reg.DESC_REGION_ABREV  ="No" THEN "Sin Información" 
    WHEN reg.DESC_REGION_ABREV = "XIII" THEN "Región Metropolitana"
    ELSE "Otras Regiones" 
END AS DESC_AGRUP_REGION_COLEGIO,
CASE WHEN f.COD_NIVEL_EDUCACIONAL_MADRE IN(8,9,21) or f.COD_NIVEL_EDUCACIONAL_PADRE in(8,9,21)  then "No Primera Generación" 
     WHEN f.COD_NIVEL_EDUCACIONAL_MADRE = 0 and f.COD_NIVEL_EDUCACIONAL_PADRE = 0 then "Sin Información" 
     WHEN f.COD_NIVEL_EDUCACIONAL_MADRE is null and f.COD_NIVEL_EDUCACIONAL_PADRE is null then "Sin Información" 
     ELSE "Primera Generación" 
END AS DESC_GEN_UNIVERSITARIA_FAMILIAR,
ramed.NOM_PARAM AS DESC_RAMA_EDUCACIONAL,
case  
    when  f.ptje_pond_selec >= 500 and f.ptje_pond_selec < 520  then "[500-520)"
    when  f.ptje_pond_selec >= 520 and f.ptje_pond_selec < 540  then "[520-540)"
    when  f.ptje_pond_selec >= 540 and f.ptje_pond_selec < 560  then "[540-560)"
    when  f.ptje_pond_selec >= 560 and f.ptje_pond_selec < 580  then "[560-580)"
    when  f.ptje_pond_selec >= 580 and f.ptje_pond_selec < 600  then "[580-600)"
    when  f.ptje_pond_selec >= 600 and f.ptje_pond_selec < 620  then "[600-620)"
    when  f.ptje_pond_selec >= 620 and f.ptje_pond_selec < 640  then "[620-640)"
    when  f.ptje_pond_selec >= 640 and f.ptje_pond_selec < 660  then "[640-660)"
    when  f.ptje_pond_selec >= 660 and f.ptje_pond_selec < 680  then "[660-680)"
    when  f.ptje_pond_selec >= 680 and f.ptje_pond_selec < 700  then "[680-700)"
    when  f.ptje_pond_selec >= 700 and f.ptje_pond_selec < 720  then "[700-720)"
    when  f.ptje_pond_selec >= 720 and f.ptje_pond_selec < 740  then "[720-740)"
    when  f.ptje_pond_selec >= 740 and f.ptje_pond_selec < 760  then "[740-760)"
    when  f.ptje_pond_selec >= 760 and f.ptje_pond_selec < 780  then "[760-780)"
    when  f.ptje_pond_selec >= 780 and f.ptje_pond_selec < 800  then "[780-800)"
    when  f.ptje_pond_selec >= 800 and f.ptje_pond_selec < 820  then "[800-820)"
    when  f.ptje_pond_selec >= 820 and f.ptje_pond_selec < 840  then "[820-840)"
    when  f.ptje_pond_selec >= 840 and f.ptje_pond_selec < 860  then "[840-860)"
    when  f.ptje_pond_selec >= 860 and f.ptje_pond_selec < 880  then "[860-880)"
    when  f.ptje_pond_selec >= 880 and f.ptje_pond_selec < 900  then "[880-900)"
    when  f.ptje_pond_selec >= 900 and f.ptje_pond_selec < 920  then "[900-920)"
    when  f.ptje_pond_selec >= 920 and f.ptje_pond_selec < 940  then "[920-940)"
    when  f.ptje_pond_selec >= 940 and f.ptje_pond_selec < 960  then "[940-960)"
    when  f.ptje_pond_selec >= 960 and f.ptje_pond_selec < 980  then "[960-980)"
    when  f.ptje_pond_selec >= 980 then "980 o más" 
    else "Sin Información"
end TRAMO_PUNTAJE_PONDERADO
from silver.adm_ficha_alumno f
left join silver.mst_trv_alumno a on f.COD_ALUMNO =a.COD_ALUMNO
left join silver.mst_trv_param via_ad on via_ad.COD_PARAM = a.COD_NOM_VIA_ADMISION and via_ad.TIPO_PARAM = 157
left join silver.mst_trv_param det_via on det_via.COD_PARAM = a.COD_DETALLE_VIA_ADMISION and det_via.TIPO_PARAM = 158
left join silver.mst_trv_curriculum cu on cu.COD_CURRICULUM = a.COD_CURRICULUM
left join silver.mst_trv_carrera car on car.COD_CARRERA = cu.COD_CARRERA
left join silver.mst_adm_colegio cole on f.ID_COLEGIO = cole.ID_COLEGIO
left join silver.mst_trv_unidad_academica unac on car.COD_UNIDAD_ACADEM = unac.COD_UNIDAD_ACADEM
left join silver.hist_carrera hc on hc.COD_CURRICULUM = a.COD_CURRICULUM and hc.ANIO_ADMIS = a.ANIO_ADMISION
left join silver.mst_trv_param deped on deped.COD_PARAM = cole.COD_TIPO_FINANCIAMIENTO and deped.TIPO_PARAM = 3
left join silver.mst_trv_geo_region reg on reg.cod_region = cole.COD_REGION
left join silver.mst_trv_param ramed on ramed.COD_PARAM = cole.COD_TIPO_EDUCACION and ramed.TIPO_PARAM = 1
                             """
d_table = spark.sql(query_q)

In [0]:
#aqui se declaran de manera standar los campos necesarios para la generacion de los logs de metadata y lineage
entity_name = tbl
applicant_id = "ADMISION_GOLD"
description = "Carga de tabla para modelo gold de admision"
business_domain = "Admision"
business_subdomain = "Admision"
business_usability = "Analisis and reporting"
business_engineer = [""]
business_owner = [""]
business_steward = [""]
business_origin_systems = ["ADMISION", "CORPORATIVO", "REGACAD", "UC_BANNER"]
business_sensitive_columns = [""]
step_stageId = stage_id
step_stage_zone = zone
step_stage_description = "Carga en capa "+zone+" de la tabla "+entity_name
step_extract_description = "Proceso de carga de tabla "+gold_tbl_name
step_extract_records = d_table.count()
step_extract_viewname = gold_tbl_name
step_extract_sourceFile = "N/A"
step_transform_description = step_extract_description
step_transform_query = query_q
step_load_description = step_extract_description
step_load_records = d_table.count()

In [0]:
%sql
CREATE SCHEMA  IF NOT EXISTS  gold

In [0]:
delta_tbl = gold_routhe+gold_tbl_name
d_table.write.format("delta").mode("overwrite").option("overwriteSchema", "true") \
    .option("path", delta_tbl).saveAsTable(schema+gold_tbl_name)

In [0]:
#principalmente variables de termino de ejecucion
end_process = datetime.now()
end_process_time = end_process.strftime("%Y-%m-%d %H:%M:%S")
dtime = start_process - end_process
elapsed_seconds_time = str(int(dtime.total_seconds()))

In [0]:
step_load_destination_file = delta_tbl
step_load_destination_table = schema+gold_tbl_name

In [0]:
metadata = spark.createDataFrame([( 
                                   applicant_id, 
                                   description, 
                                   entity_name, 
                                   start_process_time, 
                                   end_process_time, 
                                   "snapshot", 
                                   "snapshot", 
                                   business_domain, 
                                   business_subdomain, 
                                   business_usability, 
                                   business_engineer, 
                                   business_owner, 
                                   business_steward, 
                                   business_origin_systems, 
                                   business_sensitive_columns)],
                                        ["applicant_id", 
                                         "description", 
                                         "entity_name", 
                                         "start_process_time", 
                                         "end_process_time", 
                                         "frecuency", 
                                         "ingestion_type", 
                                         "business_domain", 
                                         "business_subdomain", 
                                         "business_usability", 
                                         "business_engineer", 
                                         "business_owner", 
                                         "business_steward", 
                                         "business_origin_systems", 
                                         "business_sensitive_columns"])

lineage = spark.createDataFrame([( 
                                  entity_name, 
                                  start_process_time, 
                                  end_process_time, 
                                  step_stageId, 
                                  step_stage_zone, 
                                  step_stage_description, 
                                  step_extract_description, 
                                  step_extract_records, 
                                  step_extract_viewname, 
                                  step_extract_sourceFile, 
                                  step_transform_description, 
                                  step_transform_query, 
                                  step_load_description, 
                                  step_load_records, 
                                  step_load_destination_file, 
                                  step_load_destination_table)],
                                    ["entity_name", 
                                     "start_process_time", 
                                     "end_process_time", 
                                     "step_stageId", 
                                     "step_stage_zone", 
                                     "step_stage_description", 
                                     "step_extract_description", 
                                     "step_extract_records", 
                                     "step_extract_viewname", 
                                     "step_extract_sourceFile", 
                                     "step_transform_description", 
                                     "step_transform_query", 
                                     "step_load_description", 
                                     "step_load_records", 
                                     "step_load_destination_file", 
                                     "step_load_destination_table"])

In [0]:
metadata.write.format("delta").mode("append").option("path", log_routhe+"/metadata.delta").saveAsTable("metadata."+step_stage_zone.lower()+"_"+gold_tbl_name)
lineage.write.format("delta").mode("append").option("path", log_routhe+"/lineage.delta").saveAsTable("lineage."+step_stage_zone.lower()+"_"+gold_tbl_name)
dbutils.notebook.exit([step_extract_records, step_load_records])